In [3]:
!pip install requests

In [7]:
import os
import requests
from time import sleep
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Set up the folder to store images
folder_name = "restored_images1"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# API details
api_key = "hylepsentrog"  # Replace with your actual API key
base_url = "https://api.europeana.eu/record/v2/search.json"

# Define the query parameters
params = {
    "query": "places",
    "media": "true",  # Ensures that only records with media (images) are returned
    "rows": 100,  # Number of results per page (adjust as needed)
    "start": 1,   # Start index for pagination
    "wskey": api_key  # Your API key
}

# Function to download an image
def download_image(url, image_name):
    if isinstance(url, list):
        url = url[0]  # Extract the string from the list if needed

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    session.mount('http://', HTTPAdapter(max_retries=retries))

    try:
        # Disable SSL verification if needed
        img_response = session.get(url, verify=False)
        if img_response.status_code == 200:
            with open(os.path.join(folder_name, image_name), 'wb') as img_file:
                img_file.write(img_response.content)
            print(f"Downloaded {image_name}")
        elif img_response.status_code == 404:
            print(f"Image not found (404) for {image_name}")
        else:
            print(f"Failed to download {image_name}, status code: {img_response.status_code}")
    except requests.exceptions.SSLError as e:
        print(f"SSL error occurred while downloading {image_name}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Request error occurred while downloading {image_name}: {e}")

# Fetch and process the data from the API
image_count = 277
while True:
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        items = data.get('items', [])

        # Download each image
        for item in items:
            if 'edmIsShownBy' in item:
                img_url = item['edmIsShownBy']
                download_image(img_url, f"R{image_count}.jpg")
                image_count += 1

        # Check if there are more results to fetch
        if len(items) < params['rows']:
            break  # Exit the loop if fewer items are returned than requested (end of results)
        else:
            params['start'] += params['rows']  # Move to the next page
    else:
        print(f"Failed to retrieve data from the API. Status code: {response.status_code}")
        break


Failed to retrieve data from the API. Status code: 400


another api key for places provided by the Swedish Open Cultural Heritage


In [8]:
import os
import requests
from urllib.parse import urlparse, parse_qs
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Set up the folder to store images on your local machine
folder_name = "restored_images"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Parse the provided URL to extract the base URL and query parameters
api_url = "https://api.europeana.eu/record/search.json?wskey=hylepsentrog&qf=collection%3Aphotography&qf=TYPE%3A%22IMAGE%22&qf=contentTier%3A%224%22&qf=PROVIDER%3A%22Swedish+Open+Cultural+Heritage%22&query=places&reusability=open&profile=facets&rows=0&facet=PROVIDER&f.PROVIDER.facet.limit=125000&start=1"

# Parse the URL
parsed_url = urlparse(api_url)
base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
params = parse_qs(parsed_url.query)

# Convert the list of query values to a single value where appropriate
for key in params:
    if isinstance(params[key], list) and len(params[key]) == 1:
        params[key] = params[key][0]

# Set rows to a specific value to get actual data (instead of 0)
params['rows'] = 100  # Adjust this as needed

# Function to download an image
def download_image(url, image_name):
    if isinstance(url, list):
        url = url[0]  # Extract the string from the list if needed

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    session.mount('http://', HTTPAdapter(max_retries=retries))

    try:
        # Disable SSL verification if needed
        img_response = session.get(url, verify=False)
        if img_response.status_code == 200:
            # Save the image to the local "restored_images" folder
            with open(os.path.join(folder_name, image_name), 'wb') as img_file:
                img_file.write(img_response.content)
            print(f"Downloaded {image_name}")
        elif img_response.status_code == 404:
            print(f"Image not found (404) for {image_name}")
        else:
            print(f"Failed to download {image_name}, status code: {img_response.status_code}")
    except requests.exceptions.SSLError as e:
        print(f"SSL error occurred while downloading {image_name}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Request error occurred while downloading {image_name}: {e}")

# Fetch and process the data from the API
image_count = 0
while True:
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        items = data.get('items', [])

        # Download each image
        for item in items:
            if 'edmIsShownBy' in item:
                img_url = item['edmIsShownBy']
                download_image(img_url, f"R{image_count}.jpg")
                image_count += 1

        # Check if there are more results to fetch
        if len(items) < int(params['rows']):
            break  # Exit the loop if fewer items are returned than requested (end of results)
        else:
            params['start'] = int(params['start']) + int(params['rows'])  # Move to the next page
    else:
        print(f"Failed to retrieve data from the API. Status code: {response.status_code}")
        break


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mm.dimu.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R0.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mm.dimu.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R1.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mm.dimu.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Downloaded R321.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'samlingar.goteborgsstadsmuseum.se'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R322.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mm.dimu.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R323.jpg
Downloaded R324.jpg
Downloaded R325.jpg
Downloaded R326.jpg
Downloaded R327.jpg


In [10]:
# prompt: download restored_images on my machine

from google.colab import files
!zip -r /content/restored_images.zip /content/restored_images
files.download("/content/restored_images.zip")


  adding: content/restored_images/ (stored 0%)
  adding: content/restored_images/R81.jpg (deflated 2%)
  adding: content/restored_images/R267.jpg (deflated 2%)
  adding: content/restored_images/R246.jpg (deflated 2%)
  adding: content/restored_images/R56.jpg (deflated 2%)
  adding: content/restored_images/R167.jpg (deflated 2%)
  adding: content/restored_images/R325.jpg (deflated 0%)
  adding: content/restored_images/R69.jpg (deflated 2%)
  adding: content/restored_images/R277.jpg (deflated 2%)
  adding: content/restored_images/R38.jpg (deflated 1%)
  adding: content/restored_images/R117.jpg (deflated 2%)
  adding: content/restored_images/R15.jpg (deflated 1%)
  adding: content/restored_images/R112.jpg (deflated 2%)
  adding: content/restored_images/R155.jpg (deflated 2%)
  adding: content/restored_images/R172.jpg (deflated 2%)
  adding: content/restored_images/R302.jpg (deflated 2%)
  adding: content/restored_images/R119.jpg (deflated 2%)
  adding: content/restored_images/R294.jpg (de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

another api url to get photographs collected by PHOTOCONSORTIUM

In [12]:
import os
import requests
from urllib.parse import urlparse, parse_qs
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Set up the folder to store images on your local machine
folder_name = "restored_images1"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Parse the provided URL to extract the base URL and query parameters
api_url = "https://api.europeana.eu/record/search.json?wskey=hylepsentrog&qf=collection%3Aphotography&qf=TYPE%3A%22IMAGE%22&qf=contentTier%3A%224%22&qf=PROVIDER%3A%22PHOTOCONSORTIUM%22&query=places&reusability=open&profile=facets&rows=0&facet=REUSABILITY&start=1"

# Parse the URL
parsed_url = urlparse(api_url)
base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
params = parse_qs(parsed_url.query)

# Convert the list of query values to a single value where appropriate
for key in params:
    if isinstance(params[key], list) and len(params[key]) == 1:
        params[key] = params[key][0]

# Set rows to a specific value to get actual data (instead of 0)
params['rows'] = 100  # Adjust this as needed

# Function to download an image
def download_image(url, image_name):
    if isinstance(url, list):
        url = url[0]  # Extract the string from the list if needed

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    session.mount('http://', HTTPAdapter(max_retries=retries))

    try:
        # Disable SSL verification if needed
        img_response = session.get(url, verify=False)
        if img_response.status_code == 200:
            # Save the image to the local "restored_images" folder
            with open(os.path.join(folder_name, image_name), 'wb') as img_file:
                img_file.write(img_response.content)
            print(f"Downloaded {image_name}")
        elif img_response.status_code == 404:
            print(f"Image not found (404) for {image_name}")
        else:
            print(f"Failed to download {image_name}, status code: {img_response.status_code}")
    except requests.exceptions.SSLError as e:
        print(f"SSL error occurred while downloading {image_name}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Request error occurred while downloading {image_name}: {e}")

# Fetch and process the data from the API
image_count = 0
while True:
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        items = data.get('items', [])

        # Download each image
        for item in items:
            if 'edmIsShownBy' in item:
                img_url = item['edmIsShownBy']
                download_image(img_url, f"R{image_count}.jpg")
                image_count += 1

        # Check if there are more results to fetch
        if len(items) < int(params['rows']):
            break  # Exit the loop if fewer items are returned than requested (end of results)
        else:
            params['start'] = int(params['start']) + int(params['rows'])  # Move to the next page
    else:
        print(f"Failed to retrieve data from the API. Status code: {response.status_code}")
        break


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sgdap.girona.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R0.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sgdap.girona.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R1.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sgdap.girona.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

Downloaded R289.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sgdap.girona.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R290.jpg


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sgdap.girona.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded R291.jpg


In [13]:
from google.colab import files
!zip -r /content/restored_images1.zip /content/restored_images1
files.download("/content/restored_images1.zip")


  adding: content/restored_images1/ (stored 0%)
  adding: content/restored_images1/R81.jpg (deflated 5%)
  adding: content/restored_images1/R267.jpg (deflated 4%)
  adding: content/restored_images1/R246.jpg (deflated 8%)
  adding: content/restored_images1/R56.jpg (deflated 4%)
  adding: content/restored_images1/R167.jpg (deflated 3%)
  adding: content/restored_images1/R69.jpg (deflated 5%)
  adding: content/restored_images1/R277.jpg (deflated 3%)
  adding: content/restored_images1/R38.jpg (deflated 4%)
  adding: content/restored_images1/R117.jpg (deflated 8%)
  adding: content/restored_images1/R15.jpg (deflated 7%)
  adding: content/restored_images1/R112.jpg (deflated 9%)
  adding: content/restored_images1/R155.jpg (deflated 9%)
  adding: content/restored_images1/R172.jpg (deflated 3%)
  adding: content/restored_images1/R119.jpg (deflated 4%)
  adding: content/restored_images1/R83.jpg (deflated 6%)
  adding: content/restored_images1/R25.jpg (deflated 6%)
  adding: content/restored_imag

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>